## Import Libraries

In [29]:
import numpy as np
import pandas as pd
from numpy.linalg import norm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import time
import math
import pdb

## Data reading and exploration

In [30]:
data_path = "C:/Users/Christina/Desktop/ΜΕΤΑΠΤΥΧΙΑΚΟ/βελτιστοποίηση/PROJECT-2/PROJECT2/Βελτιστοποίηση/Data/breast-cancer.data"

In [31]:
column_names = ["Class", "Age", "Menopause", "Tumor-size", "Inv-nodes", "Node-caps", "Deg-malig", "Breast", "Breast-quad", "Irradiat"]
df = pd.read_csv(data_path, names=column_names, header=None, na_values="?")

In [32]:
df.head()

,Class,Age,Menopause,Tumor-size,Inv-nodes,Node-caps,Deg-malig,Breast,Breast-quad,Irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class        286 non-null    object
 1   Age          286 non-null    object
 2   Menopause    286 non-null    object
 3   Tumor-size   286 non-null    object
 4   Inv-nodes    286 non-null    object
 5   Node-caps    278 non-null    object
 6   Deg-malig    286 non-null    int64 
 7   Breast       286 non-null    object
 8   Breast-quad  285 non-null    object
 9   Irradiat     286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


## Drop the data with missing values

In [34]:
df.dropna(inplace=True)

## Split the data from labels

In [35]:
data = df[["Age", "Menopause", "Tumor-size", "Inv-nodes", "Node-caps", "Deg-malig", "Breast", "Breast-quad", "Irradiat"]]
labels = df[["Class"]]
del df

## Use one-hot encoding for categorical features

In [36]:
# One-hot encoding
categorical_features = ["Age", "Menopause", "Tumor-size", "Inv-nodes", "Node-caps", "Breast", "Breast-quad", "Irradiat"]
data = pd.get_dummies(data, columns=categorical_features)
labels = pd.get_dummies(labels, columns=["Class"])

In [37]:
data.head()

,Deg-malig,Age_20-29,Age_30-39,Age_40-49,Age_50-59,Age_60-69,Age_70-79,Menopause_ge40,Menopause_lt40,Menopause_premeno,...,Node-caps_yes,Breast_left,Breast_right,Breast-quad_central,Breast-quad_left_low,Breast-quad_left_up,Breast-quad_right_low,Breast-quad_right_up,Irradiat_no,Irradiat_yes
0,3,0,1,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,1,0
1,2,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
2,2,0,0,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,1,0
3,2,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
4,2,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 285
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Deg-malig              277 non-null    int64
 1   Age_20-29              277 non-null    uint8
 2   Age_30-39              277 non-null    uint8
 3   Age_40-49              277 non-null    uint8
 4   Age_50-59              277 non-null    uint8
 5   Age_60-69              277 non-null    uint8
 6   Age_70-79              277 non-null    uint8
 7   Menopause_ge40         277 non-null    uint8
 8   Menopause_lt40         277 non-null    uint8
 9   Menopause_premeno      277 non-null    uint8
 10  Tumor-size_0-4         277 non-null    uint8
 11  Tumor-size_10-14       277 non-null    uint8
 12  Tumor-size_15-19       277 non-null    uint8
 13  Tumor-size_20-24       277 non-null    uint8
 14  Tumor-size_25-29       277 non-null    uint8
 15  Tumor-size_30-34       277 non-null    u

In [10]:
labels.head()

,Class_no-recurrence-events,Class_recurrence-events
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


## Convert dataframes to numpy arrays

In [22]:
data = data.to_numpy()
labels = labels.to_numpy()
#print(data.shape, labels.shape)
#print(labels[0])

## Scale the data features

In [23]:
standard_scaler = StandardScaler()
data = standard_scaler.fit_transform(data)

In [13]:
#data_ordering = np.random.permutation(data.shape[0])
#data = data[data_ordering]
#labels = labels[data_ordering]

#index_cutoff = 100
#data = data[ : index_cutoff]
#labels = labels[ : index_cutoff]

#count = 0
#for label in labels:
#    if label[0] == 0:
#        count += 1
#print(count)
#print(data[0])

In [24]:
class Optimizer:
    def __init__(self, neural_network, max_iterations=1000):
        self.nn = neural_network
        self.max_iterations = max_iterations
        
    def random_search_gaussian_distribution(self, model_parameters, sigma=0.5):
        iteration = 0
        x_best = model_parameters
        iteration += 1
        x_best_fvalue = self.nn.error_function(x_best)
        mu = x_best
        
        print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
        while (iteration < self.max_iterations):
            # Sample a new point
            xk = np.random.normal(mu, sigma, size=model_parameters.shape)
            iteration += 1
            xk_fvalue = self.nn.error_function(xk)

            if(xk_fvalue < x_best_fvalue):
                mu, x_best = xk, xk
                x_best_fvalue = xk_fvalue
                print("Iteration=%d, Error=%.10f !!!" % (iteration, x_best_fvalue))
            else:
                print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
                
        return x_best, x_best_fvalue
    
    def random_walk_variable_step(self, model_parameters, s=1, a=1.5, beta=0.5):
        iteration = 0
        x = model_parameters
        x_best = model_parameters
        
        iteration += 1
        x_fvalue = self.nn.error_function(x)
        x_best_fvalue = x_fvalue
        
        print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
        while (iteration < self.max_iterations):
            # Pick a random direction
            p = np.random.uniform(-1, 1, size=model_parameters.shape)
            pu = p / norm(p, 1)
            # Calculate xt
            xt = x + s * pu
            
            iteration += 1
            xt_fvalue = self.nn.error_function(xt)

            if (xt_fvalue < x_fvalue):
                # make step bigger
                sl = a * s

                # try bigger step
                xl = x + sl * pu
                
                iteration += 1
                xl_fvalue = self.nn.error_function(xl)

                if (xl_fvalue < xt_fvalue):
                    x = xl
                    s = sl
                    x_fvalue = xl_fvalue
                else:
                    x = xt
                    x_fvalue = xt_fvalue
            else:
                # make step smaller
                sm = beta * s

                # try smaller step
                xm = x + sm * pu
                
                iteration += 1
                xm_fvalue = self.nn.error_function(xm)

                if (xm_fvalue < x_fvalue):
                    x = xm
                    s = sm
                    x_fvalue = xm_fvalue

            # Update best
            if (x_fvalue < x_best_fvalue):
                x_best = x
                x_best_fvalue = x_fvalue

                print("Iteration=%d, Error=%.10f !!!" % (iteration, x_best_fvalue))
            else:
                print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))

        return x_best, x_best_fvalue
    
    def simulated_annealing(self, model_parameters, t=1, a=1, b=1, r=0.5):
        iteration = 0
        x = model_parameters
        x_fvalue = self.nn.error_function(x)

        x_best = model_parameters
        iteration += 1
        x_best_fvalue = self.nn.error_function(x_best)
        
        # The smallest value the error function can possible get
        f_star = 0
        
        # Metropolis Function
        Facc = lambda x_fvalue, y_fvalue, t: min(1, math.exp(-(y_fvalue - x_fvalue) / t))
        # Cooling method
        U = lambda a, b, x_fvalue, f_star: b * math.pow((x_fvalue - f_star), a)
        
        print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
        while (iteration < self.max_iterations):
            # Pick a random direction
            s = np.random.uniform(-1, 1, size=model_parameters.shape)
            s = s / norm(s, 1)
            # Calculate y
            y = x + r * s
            
            iteration += 2
            x_fvalue = self.nn.error_function(x)
            y_fvalue = self.nn.error_function(y)
            
            Facc_k = Facc(x_fvalue, y_fvalue, t)

            if (np.random.uniform(0, 1) < Facc_k):
                x = y
                x_fvalue = y_fvalue

            t = U(a, b, x_fvalue, f_star)

            if (x_fvalue < x_best_fvalue):
                x_best = x
                x_best_fvalue = x_fvalue
                print("Iteration=%d, Error=%.10f !!!" % (iteration, x_best_fvalue))
            else:
                print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
                
        return x_best, x_best_fvalue
    
    def Nelder_Mead(self, model_parameters):
        iteration = 0
        simplex = list()
        
        # ro operators
        ro_reflection = 1
        ro_expansion = 2 * ro_reflection
        ro_external_contraction = 0.5 * ro_reflection
        ro_internal_contraction = -0.5
        
        iteration += 1
        # simplex is a list like that list([x1, fvalue_x1], ..., [x_n+1, fvalue_x_n+1])
        simplex.append([model_parameters, self.nn.error_function(model_parameters)])
        simplex_dimention = model_parameters.shape[0] + 1
        print("Iteration=%d, Error=%.10f" % (iteration, simplex[0][1]))
        
        # create n random x, fvalue_x pairs and add them to simplex
        for i in range(1, simplex_dimention):
            xi = np.random.uniform(0, math.sqrt(5), size=model_parameters.shape)
            iteration += 1
            simplex.append([xi, self.nn.error_function(xi)]) 

        # This function creates a new point based on ro operator, the worst
        # x value and the x_centroid
        point_generator = lambda x_centroid, x_worst, ro: (1 + ro) * x_centroid - ro * x_worst
        
        [
        [x0,fx0] 0
        [x1,fx1] 1
        ...
        ] #simplex. 
        
        x_best = simplex[0][0] #apo to 0 pare to 0 pou anaferetai sto x0
        x_best_fvalue = simplex[0][1]  #kai to 1 pou anaferetai sto fx0
        
        while (iteration < self.max_iterations):
            operator_activated = False
            # Sort simplex
            simplex = sorted(simplex, key=lambda x:x[1])
            
            # Calculate centroid
            x_centroid = np.zeros(shape=model_parameters.shape)
            for i in range(0, simplex_dimention - 1):
                x_centroid = np.add(x_centroid, simplex[i][0])
            x_centroid = x_centroid / x_centroid.shape[0]

            # Reflection
            iteration += 1
            x_reflection = point_generator(x_centroid, simplex[-1][0], ro_reflection)

            x_reflection_fvalue = self.nn.error_function(x_reflection)
            x_worst_fvalue = simplex[-1][1]
            x_best_fvalue = simplex[0][1]
            x_best = simplex[0][0]
            
            if (x_best_fvalue <= x_reflection_fvalue < x_worst_fvalue):
                simplex[-1] = [x_reflection, x_reflection_fvalue]
                operator_activated = True

            # Expansion
            if (x_reflection_fvalue < x_best_fvalue):
                x_expansion = point_generator(x_centroid, simplex[-1][0], ro_expansion)
                iteration += 1
                x_expansion_fvalue = self.nn.error_function(x_expansion)
                if (x_expansion_fvalue < x_reflection_fvalue):
                    simplex[-1] = [x_expansion, x_expansion_fvalue]
                    operator_activated = True
                else:
                    simplex[-1] = [x_reflection, x_reflection_fvalue]
                    operator_activated = True

            # External constraction
            x_second_worst_fvalue = simplex[-2][1]
            if (x_second_worst_fvalue <= x_reflection_fvalue < x_worst_fvalue):
                x_external_contraction = point_generator(x_centroid, simplex[-1][0], ro_external_contraction)
                iteration += 1
                x_external_contraction_fvalue = self.nn.error_function(x_external_contraction)
                if (x_external_contraction_fvalue <= x_reflection_fvalue):
                    simplex[-1] = [x_external_contraction, x_external_contraction_fvalue]
                    operator_activated = True

            # Internal Contraction
            if (x_worst_fvalue < x_reflection_fvalue):
                x_internal_contraction = point_generator(x_centroid, simplex[-1][0], ro_internal_contraction)
                iteration += 1
                x_internal_contraction_fvalue = self.nn.error_function(x_internal_contraction)
                if (x_internal_contraction_fvalue < x_worst_fvalue):
                    simplex[-1] = [x_internal_contraction, x_internal_contraction_fvalue]
                    operator_activated = True
                
            # Shrink Simplex towards best x
            if (not operator_activated):
                for i in range(1, simplex_dimention):
                    simplex[i][0] = simplex[0][0] - ((simplex[i][0] - simplex[0][0]) / 2)
            
            print("Iteration=%d, Error=%.10f" % (iteration, x_best_fvalue))
          
        return x_best, x_best_fvalue

## Probabilistic Neural  Network

In [1]:
class Neuron:
    def __init__(self, x):
        self.center = x
        self.n = x.shape[0]
        self.n_div_two = self.n / 2
        self.pi_mul_two = 2 * math.pi

    def gaussian_activation(self, x, det_spr_par_mat, inv_model_parameters, inv_spr_par_mat):
        vector = np.subtract(x, self.center)
        activation = 1 / (math.pow(self.pi_mul_two, self.n_div_two) * math.sqrt(det_spr_par_mat))
        activation *= math.exp((-1/2) * (np.transpose(vector) @ inv_spr_par_mat @ vector)) 
        return activation
    
class Neural_Network:
    def __init__(self, data, labels):
        self.neuros_class_1 = list()
        self.neuros_class_2 = list()
        self.data = data
        self.labels = labels
        self.parameters_shape = data.shape[1]
        self.__init_neurons()

    def __init_neurons(self):
        # Create neurons with center data_i
        for index, data_i in enumerate(self.data):
            # Create a neuron with center the data_i
            neuron = Neuron(data_i)
            # If data is category one the neuron  goes to 
            # category one else in category 2
            if (self.labels[index] == np.array([1, 0])).all():
                self.neuros_class_1.append(neuron)
            else:
                self.neuros_class_2.append(neuron)
        self.neuros_class_1 = np.asarray(self.neuros_class_1)
        self.neuros_class_2 = np.asarray(self.neuros_class_2)
        
    def __forward_pass(self, model_parameters, data):
        output_vector = list()
        prior_class_1 = 1/2
        prior_class_2 = 1/2
        # sigma -> sigma^2
        model_parameters_squared = np.square(model_parameters)
        det_spr_par_mat = np.prod(model_parameters_squared)
        inv_model_parameters = np.float_power(model_parameters_squared, -1)
        inv_spr_par_mat = np.diag(inv_model_parameters)
        
        #prior_class_1 = self.neuros_class_1.shape[0] / data.shape[0]
        #prior_class_2 = self.neuros_class_2.shape[0] / data.shape[0]

        # For every data_i in dataset
        for index, data_i in enumerate(data):
            sum_class_1 = 0
            sum_class_2 = 0
            
            # For every neuron in class 1 neurons pass the data_i
            for sigma_index, neuron in enumerate(self.neuros_class_1):
                sum_class_1 += neuron.gaussian_activation(data_i, det_spr_par_mat, inv_model_parameters, inv_spr_par_mat)
            
            # For every neuron in class 2 neurons pass the data_i
            for sigma_index, neuron in enumerate(self.neuros_class_2):
                sum_class_2 += neuron.gaussian_activation(data_i, det_spr_par_mat, inv_model_parameters, inv_spr_par_mat)
                
            try:
                total_norm_sum = prior_class_2 * sum_class_2 + prior_class_1 * sum_class_1
                norm_sum_class_1 = prior_class_1 * sum_class_1 / total_norm_sum
                norm_sum_class_2 = prior_class_2 * sum_class_2 / total_norm_sum
            except:
                norm_sum_class_1 = 0.5
                norm_sum_class_2 = 0.5
            
            # Output is a probability distribution for example [0.8, 0.2]
            # So the network believes that most likely is category 1 the data_i
            output_vector.append([norm_sum_class_1, norm_sum_class_2])
            
        return np.asarray(output_vector)
    
    def error_function(self, model_parameters):
        n_data = self.data.shape[0]
        output_vector = self.__forward_pass(model_parameters, self.data)
        mean_squared_error = np.sum(np.square(norm(output_vector - self.labels, 1))) / n_data
        
        return mean_squared_error
    
    def predict(self, model_parameters, data):
        predicted_classes = list()
        output_vector = self.__forward_pass(model_parameters, data)
        
        for index, vector in enumerate(output_vector):
            # if the class 1 has higher probability classify
            # the example as [1, 0] else as [0, 1]
            if vector[1] < vector[0]:
                predicted_classes.append([1, 0])
            else:
                predicted_classes.append([0, 1])
            
        return np.asarray(predicted_classes)

In [26]:
# Timer
time_0 = time.time()
neural_network = Neural_Network(data, labels)
parameters_shape = neural_network.parameters_shape
init_parameters = np.random.uniform(low=0, high=math.sqrt(5), size=(parameters_shape))
neural_network.error_function(init_parameters)
#optimizer = Optimizer(neural_network, max_iterations=10)

#optimized_parameters, minimum_error = optimizer.random_search_gaussian_distribution(init_parameters, sigma=0.5)
#optimized_parameters, minimum_error = optimizer.random_walk_variable_step(init_parameters, s=1, a=1.5, beta=0.5)
#optimized_parameters, minimum_error = optimizer.simulated_annealing(init_parameters, t=1, a=1, b=1, r=0.5)
#optimized_parameters, minimum_error = optimizer.Nelder_Mead(init_parameters)

# Timer
time_1 = time.time()
time_difference = time_1 - time_0
print("Time in seconds: %.2f" % (time_difference))

Time in seconds: 0.93


In [17]:
# Make the predictions in test set
#predictions = neural_network.predict(optimized_parameters, data)
#print("Train set classification accuracy: %.2f" % (accuracy_score(labels, predictions)))

In [ ]:
p= pd.read_excel("C:/Users/Christina/Desktop/ΜΕΤΑΠΤΥΧΙΑΚΟ/βελτιστοποίηση/PROJECT-2/PROJECT2/Βελτιστοποίηση//mydata.xlsx", header=None)